Importing necessary libraries

In [ ]:
from skimage import color, data, io, exposure
from skimage.feature import canny, hog
import datetime
import cv2
import numpy as np
from sklearn.feature_extraction import image
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
import xlwings as xw
import sys
import math
import argparse
from distutils import util

In [ ]:
# This function that takes parameters of the project code.
def handle_args():
    parser=argparse.ArgumentParser()
    parser.add_argument('-f')
    parser.add_argument('--image_name', default='p001')
    parser.add_argument('--max_depth', default=20)
    parser.add_argument('--avg_patch_size', default=39)
    parser.add_argument('--sum_patch_size', default=8)
    parser.add_argument('--sd_patch_size', default=16)
    parser.add_argument('--hog_patch_size', default=1)
    parser.add_argument('--neigh_patch_size', default=1)
    parser.add_argument('--use_distance_features', default=True)
    parser.add_argument('--use_sd_features', default=False)
    parser.add_argument('--use_sum_features', default=True)
    parser.add_argument('--use_avg_features', default=True)
    parser.add_argument('--use_edge_features', default=False)
    parser.add_argument('--use_haris_corner', default=True)
    parser.add_argument('--use_hog_features', default=False)
    parser.add_argument('--use_fast_feature', default=True)
    parser.add_argument('--use_orb_feature', default=False)
    parser.add_argument('--use_neighborhood_features', default=False)
    parser.add_argument('--save_to_file', default=False)
    args=parser.parse_args()
    args.use_fast_feature = util.strtobool(str(args.use_fast_feature))
    args.use_orb_feature = util.strtobool(str(args.use_orb_feature))
    args.use_neighborhood_features = util.strtobool(str(args.use_neighborhood_features))
    args.use_hog_features = util.strtobool(str(args.use_hog_features))
    args.use_haris_corner = util.strtobool(str(args.use_haris_corner))
    args.use_edge_features = util.strtobool(str(args.use_edge_features))
    args.use_avg_features = util.strtobool(str(args.use_avg_features))
    args.use_sum_features = util.strtobool(str(args.use_sum_features))
    args.use_sd_features = util.strtobool(str(args.use_sd_features))
    args.use_distance_features = util.strtobool(str(args.use_distance_features))
    args.save_to_file = util.strtobool(str(args.save_to_file))

    args.avg_patch_size = int(args.avg_patch_size)
    args.sum_patch_size = int(args.sum_patch_size)
    args.sd_patch_size = int(args.sd_patch_size)
    args.hog_patch_size = int(args.hog_patch_size)
    args.neigh_patch_size = int(args.neigh_patch_size)
    args.max_depth = int(args.max_depth)
    return args

In [ ]:
# This function shows the images which is passed as parameters with a special format
def show_image(dic, image_name, save = False):
    fig, axes = plt.subplots(1, len(dic), figsize = (24,16))
    ax = axes.ravel()
    
    for i in dic:
        ax[list(dic.keys()).index(i)].set_title(i)
        ax[list(dic.keys()).index(i)].imshow(dic[i], cmap = "gray")

    fig.tight_layout()
    time = datetime.datetime.now()
    plt.savefig(f'results/{image_name}.{time.hour}.{time.minute}.png', format="png")
    if not save:
        plt.show()
    plt.close()

In [ ]:
# This function finds the last row in excel
def find_last_row_in_excel(workbooklocation,sheetname,columnletter): 
    wb = xw.Book(workbooklocation)
    X = wb.sheets[sheetname].range(columnletter + str(wb.sheets[sheetname].cells.last_cell.row)).end('up').row + 1
    cell = columnletter + str(X)
    print(cell)
    return cell



# This function writes data which is passed as parameters into excel file.
def write_data_to_excel(features, max_depth, avg_patch_size, sum_patch_size, mean_abs_err, image_name, finish_time):
    try:
        wb = xw.Book('mae.xlsx')
        sht = wb.sheets['Sheet1']

        if(image_name == "p001"):
            last_row = find_last_row_in_excel('mae.xlsx','Sheet1','A')
            sht.range(last_row).value = ['Mean Absolute Error','Image Name', 'Process_time (second)', f'{features}',f'Max Depth: {max_depth}', f'Avg PS: {avg_patch_size}', f'Sum PS: {sum_patch_size}']

        last_row = find_last_row_in_excel('MAE.xlsx','Sheet1','A')
        sht.range(last_row).value = [f'{float(mean_abs_err)}',f'{image_name}',f'{finish_time}']
    except e:
        print("The file named mae.xlsx is not found in the project directory.", e)

In [ ]:
# This function gets the source image
def get_source(image_name):
    return io.imread(f'Dataset\\{image_name}, a_source.png')

# This function gets the source image with openCV
def get_source_opencv(image_name):
    return cv2.imread(f'Dataset\\{image_name}, a_source.png')

# This function gets the target image with openCV
def get_target_opencv(image_name):
    return cv2.imread(f'Dataset\\{image_name}, b_target.png')

# This function gets the target image
def get_target(image_name):
    return io.imread(f'Dataset\\{image_name}, b_target.png')

# This function gets the groundtruth image
def get_groundtruth(image_name):
    return io.imread(f'Dataset\\{image_name}, c_groundtruth.png')

In [ ]:
# This function reconstructs an image according to rgb and dimension values which is passed as parameter
def reconstruct(r, g, b, dimensions):
    image = np.zeros((dimensions[0] * dimensions[1], 3))
    for i in range(dimensions[0] * dimensions[1]):
        image[i][0] = int(r[i])
        image[i][1] = int(g[i])
        image[i][2] = int(b[i])
    image = np.reshape(image, (dimensions[0], dimensions[1], 3))
    return image.astype(int)

In [ ]:
# This function converts the image into one dimension
def get_one_dim(image, colored = False):
    if colored:
        return np.reshape(image, (image.shape[0] * image.shape[1], 1, 3))
    else:
        return np.reshape(image, (image.shape[0] * image.shape[1], 1))

In [ ]:
# This function converts the image values which is integer into float type
def convert_float(image):
    squarer = lambda t: t / 255
    vfunc = np.vectorize(squarer)
    return vfunc(image)

In [ ]:
# This function calculates average of special pixel and their neighboors.
def get_avg(data, point, patch_size):
    first, second, third, fourth = get_quarters(data, point, patch_size)
    summation = (np.sum(first) + np.sum(second) + np.sum(third) + np.sum(fourth) + np.sum(data[point[0], point[1]]))
    length = (first.shape[0] * first.shape[1] + second.shape[0] * second.shape[1] + 
             third.shape[0] * third.shape[1] + fourth.shape[0] * fourth.shape[1] + 1)
    return summation/length

In [ ]:
# This function calculates the sum of special pixel and their neigboors.
def get_sum(data, point, patch_size):
    first, second, third, fourth = get_quarters(data, point, patch_size)
    summation = (np.sum(first) + np.sum(second) + np.sum(third) + np.sum(fourth) + np.sum(data[point[0], point[1]]))
    return summation

In [ ]:
# This function calculates the standard deviation of special pixel and their neigboors
def get_sd(data, point, patch_size):
    first, second, third, fourth = get_quarters(data, point, patch_size)
    summation = (np.sum(first) + np.sum(second) + np.sum(third) + np.sum(fourth) + np.sum(data[point[0], point[1]]))
    length = (first.shape[0] * first.shape[1] + second.shape[0] * second.shape[1] + 
             third.shape[0] * third.shape[1] + fourth.shape[0] * fourth.shape[1] + 1)
    avg = summation/length
    _sum = 0.0
    first = first.flatten()
    for i in first.flatten():
        _sum = _sum + pow((i - avg), 2)
    for i in second.flatten():
        _sum = _sum + pow((i - avg), 2)
    for i in third.flatten():
        _sum = _sum + pow((i - avg), 2)
    for i in fourth.flatten():
        _sum = _sum + pow((i - avg), 2)
    result = float(_sum/length)
    return math.sqrt(result)

In [ ]:
# This function finds the neighboorhood of special pixel according to patch_size parameter which define window size like 3x3, 5x5.  
# Then returns the quarters of this special pixel.
def get_quarters(data, point, patch_size):
    #first quarter
    x_start = point[0] - patch_size
    x_end = point[0] - 1
    y_start = point[1]
    y_end = point[1] + patch_size
    
    if x_start < 0:
        x_start = 0
    if y_end > data.shape[1]:
        y_end = data.shape[1]
    first = get_portion(data, x_start, x_end, y_start, y_end)
    
    #second quarter
    x_start = point[0] - patch_size
    x_end = point[0]
    y_start = point[1] - patch_size
    y_end = point[1] - 1
    
    if x_start < 0:
        x_start = 0
    if y_start < 0:
        y_start = 0
    second = get_portion(data, x_start, x_end, y_start, y_end)
    
    #third quarter
    x_start = point[0] + 1
    x_end = point[0] + patch_size
    y_start = point[1] - patch_size
    y_end = point[1]
    
    if x_end > data.shape[0]:
        x_end = data.shape[0]
    if y_start < 0:
        y_start = 0
    third = get_portion(data, x_start, x_end, y_start, y_end)
    
    #fourth quarter
    x_start = point[0]
    x_end = point[0] + patch_size
    y_start = point[1] + 1
    y_end = point[1] + patch_size
    
    if x_end > data.shape[0]:
        x_end = data.shape[0]
    if y_end > data.shape[1]:
        y_end = data.shape[1]
    fourth = get_portion(data, x_start, x_end, y_start, y_end)
    
    return first, second, third, fourth

In [ ]:
# This function returns the certain parts of data according to given parameters
def get_portion(data, x_start, x_end, y_start, y_end):
    return data[x_start:x_end + 1, y_start:y_end + 1]

In [ ]:
# This function creates a Random Forest Regressior, Then, doing fitting process.
def fit_channel(X, y, n_estimator = 16):
    classifier = RandomForestRegressor(n_estimators=n_estimator)
    classifier.fit(X, y)
    return classifier

# This function creates a Random Forest Regressior with Randomized Search Cross Validation to find best model parameters.
def model_with_best_parameters(X,y):
    # Number of trees in random forest
    n_estimators = [int(x) for x in np.linspace(start = 10, stop = 50, num = 5)]
    # Number of features to consider at every split
    max_features = ['auto', 'sqrt']
    # Maximum number of levels in tree
    max_depth = [int(x) for x in np.linspace(10, 50, num = 5)]
    max_depth.append(None)
    # Minimum number of samples required to split a node
    min_samples_split = [2, 5, 10]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2, 4]
    # Method of selecting samples for training each tree
    bootstrap = [True, False]
    # Create the random grid
    random_grid = {'n_estimators': n_estimators,
                   'max_features': max_features,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf,
                   'bootstrap': bootstrap}
    
    # Use the random grid to search for best hyperparameters
    # First create the base model to tune
    rf = RandomForestRegressor()
    # Random search of parameters, using 3 fold cross validation, 
    # search across 100 different combinations, and use all available cores
    rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=2, random_state=42, n_jobs = -1)
    # Fit the random search model
    rf_random.fit(X, y)
    print(rf_random.best_params_)
    
    return rf_random.best_estimator_

# This function does predict process according to given classifier.
def predict_channel(classifier, X):
    # The score method returns the accuracy of the model
    return classifier.predict(X)

# This function finds edges on image with canny
def get_edge_feature(image):
    edges = canny(image)
    return np.reshape(edges, (image.shape[0] * image.shape[1], 1))

# This function finds sum value of each pixels of their neighbor pixels
def get_sum_feature(data, patch_size):
    sum_feature = np.zeros((data.shape[0], data.shape[1]))
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            sum_feature[i][j] = get_sum(data, (i, j), patch_size)
    return np.reshape(sum_feature, (sum_feature.shape[0] * sum_feature.shape[1], 1))

# This function finds average value of each pixels of their neighbor pixels
def get_avg_feature(data, patch_size):
    avg_feature = np.zeros((data.shape[0], data.shape[1]))
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            avg_feature[i][j] = get_avg(data, (i, j), patch_size)
    return np.reshape(avg_feature, (avg_feature.shape[0] * avg_feature.shape[1], 1))

# This function finds neighborhood values of each pixels of their neighbor pixels
def get_neighborhood_feature(data, patch_size):
    neigh = np.empty((1,int((patch_size * 2 + 1)**2 - 1)))
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            feature = get_neighborhood(data, (i, j), patch_size)
            neigh = np.vstack((neigh,feature))
    neigh_deleted = np.delete(neigh, 0, 0)
    return neigh_deleted

# This function finds standard deviation of each pixels of their neighbor pixels
def get_sd_feature(data, patch_size):
    sd_feature = np.zeros((data.shape[0], data.shape[1]))
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            sd_feature[i][j] = get_sd(data, (i, j), patch_size)
    return np.reshape(sd_feature, (sd_feature.shape[0] * sd_feature.shape[1], 1))

# This function finds Histogram of Oriented Gradients (HOG) value of each pixels
def get_hog_feature(data, hog_patch_size):
    fd, hog_image = hog(data, orientations=8, pixels_per_cell=(2, 2),
                    cells_per_block=(2, 2), visualize=True, multichannel=False)

    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))

    return get_sum_feature(hog_image_rescaled, hog_patch_size)

# This function finds corners value of each pixels with using Haris Corner Detection
def get_haris_corner(image,img):
    gray_img = np.float32(image)
    dst = cv2.cornerHarris(gray_img, blockSize=2, ksize=3, k=0.04)
    dst = cv2.dilate(dst, None)
    return np.reshape(dst, (dst.shape[0]*dst.shape[1],1))

# This function finds corners value of each pixels with using FAST Corner Detection Algorithm
def get_fast_feature(gray_img, img):
    fast = cv2.FastFeatureDetector_create()
    fast.setNonmaxSuppression(False)

    kp = fast.detect(gray_img, None)
    kp_img = cv2.drawKeypoints(img, kp, None, color=(0, 255, 0))
    
    return np.reshape(kp_img, (kp_img.shape[0] * kp_img.shape[1], 3))

# This function finds key points values of each pixels with using ORB (Oriented FAST and Rotated Brief) algorithm
def get_orb_feature(gray_img, img):
    orb = cv2.ORB_create(nfeatures=2000)
    kp, des = orb.detectAndCompute(gray_img, None)

    kp_img = cv2.drawKeypoints(img, kp, None, color=(0, 255, 0), flags=0)
    
    return np.reshape(kp_img, (kp_img.shape[0] * kp_img.shape[1], 3))

# This function finds neighborhood values of special pixels of their neighbor pixels
def get_neighborhood(data, point, patch_size):
    quarter_len = int(((patch_size * 2 + 1)**2 - 1) / 4)
    first, second, third, fourth = get_quarters(data, point, patch_size)
    
    first = first.flatten()
    second = second.flatten()
    third = third.flatten()
    fourth = fourth.flatten()
    
    feature_mean = np.concatenate((first, second,third,fourth), axis=None)
    feature_mean = np.mean(feature_mean)
    if(len(first) < int(quarter_len)):
        array_first_mean = np.empty((quarter_len - len(first),)) 
        array_first_mean[:] = feature_mean

        first = np.concatenate((first,array_first_mean), axis=None)
        
    if(len(second) < int(quarter_len)):
        array_second_mean = np.empty((quarter_len - len(second),)) 
        array_second_mean[:] = feature_mean

        second = np.concatenate((second,array_second_mean), axis=None)
       
    if(len(third) < int(quarter_len)):
        array_third_mean = np.empty((quarter_len - len(third),)) 
        array_third_mean[:] = feature_mean

        third = np.concatenate((third,array_third_mean), axis=None)
    
    if(len(fourth) < int(quarter_len)):
        array_fourth_mean = np.empty((quarter_len - len(fourth),)) 
        array_fourth_mean[:] = feature_mean
        
        fourth = np.concatenate((fourth,array_fourth_mean), axis=None)
       
    features = np.concatenate((first, second,third,fourth), axis=None)
    return features

# This function is helper function for doing called function as generic 
def get_feature(func, image_name, X, X_test):
    source_opencv = get_source_opencv(image_name)
    source_gray_2d_opencv = cv2.cvtColor(source_opencv, cv2.COLOR_BGR2GRAY)
    
    feature_x = func(source_gray_2d_opencv, source_opencv)
    result_x = np.hstack((X,feature_x))
    
    target_opencv = get_target_opencv(image_name)
    target_opencv = cv2.cvtColor(target_opencv, cv2.COLOR_BGR2GRAY)
    
    feature_test = func(target_opencv,target_opencv)
    result_test = np.hstack((X_test, feature_test))
    
    return result_x, result_test

# This function finds each pixels distance to top,bottom,right and left distance
def get_distance_feature(data):
    distance_down_feature = np.zeros((data.shape[0], data.shape[1]))
    distance_up_feature = np.zeros((data.shape[0], data.shape[1]))
    distance_left_feature = np.zeros((data.shape[0], data.shape[1]))
    distance_right_feature = np.zeros((data.shape[0], data.shape[1]))
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            distance_up_feature[i][j] = i
            distance_down_feature[i][j] = data.shape[0] - i
            distance_left_feature[i][j] = j
            distance_right_feature[i][j] = data.shape[1] - j
    return np.reshape(distance_up_feature, (data.shape[0] * data.shape[1], 1)), np.reshape(distance_down_feature, (data.shape[0] * data.shape[1], 1)), np.reshape(distance_left_feature, (data.shape[0] * data.shape[1], 1)), np.reshape(distance_right_feature, (data.shape[0] * data.shape[1], 1))